# Projetando um PID

## Modelando o sistema

Este trabalho foi baseado no problema 3.17 do livro *Control Systems* de *Norman S. Nise*, Sétima edição.

![Modelando o vôo do míssil](img/missil.png)

O problema modela o vôo de um míssil, que está sujeito a quatro forças: empuxo (*thrust*), sustentação (*lift*),
arrasto (*drag*) e gravidade. O míssil voa com um ângulo de ataque, $\alpha$, do seu eixo longitudinal, criando sustentação. Para seguir um determinado rumo, o ângulo do corpo da vertical, $\phi$, é controlado rotacionando o motor na cauda. 

A função de transferência relaciona o ângulo do corpo, $\phi$, e sua posição angular, $\delta$, do motor na forma, como mostrado na equação abaixo:

\begin{align}
 \frac{\Phi(s)}{\delta(s)} = \frac{K_a s + K_b}{K_3 s^3 + K_2 s^2 + K_1 s + K_0} \label{eq:plant}
\end{align}

## Entendendo a planta

Desejamos projetar um controlador PID para controlar o míssil, conforme relacionado pelo diagrama de blocos abaixo:

![Diagrama de blocos](img/block-diagram.svg)

Para fins práticos, escolheremos os parâmetros da planta de forma arbitrária.

A resposta ao degrau e impulso da planta são mostrados abaixo:

In [103]:
%matplotlib ipympl
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d
from control import (TransferFunction, step_response, bode_plot,
                     impulse_response, series, feedback, rlocus,
                     margin, nyquist_plot)

ka, kb,  = [1, 5]
k3, k2, k1, k0 = [2, 50, 10, 5]

plant_tf = TransferFunction([ka, kb], [k3, k2, k1, k0])

In [42]:
def plot_step_response(tf, title='Step response', show_points=False):
    time, output = step_response(tf)
    fig = plt.figure()
    plt.plot(time, output)
    if show_points:
        plt.plot(time, output,'om')
    plt.title(title)
    plt.show()
    return time,output

time,output = plot_step_response(plant_tf, 
                                 title='Plant step response')

FigureCanvasNbAgg()

E também podemos visualizar a resposta em frequência da planta através do diagrama de *Bode*:

In [3]:
plt.figure()
plt.title("Bode plot")
mag, phase, omega = bode_plot(plant_tf)

FigureCanvasNbAgg()

## Ajustando controlador

### Método resposta a frequência de Ziegler-Nichols
A fim de aplicarmos este método, acharemos o ganho crítico do sistema. Para isso analisaremos o lugar das raízes:

In [4]:
loci = rlocus(plant_tf, Plot=True, PrintGain=True)

FigureCanvasNbAgg()

Porém, percebe-se que nosso sistema não tem um ganho crítico, podemos aumentá-lo indefinidamente. Logo, não podemos utilizar este método.

### Método resposta ao degrau de Ziegler-Nichols

Para este método, precisamos achar o ponto de inflexão da curva para podermos calcular dois parâmetros: $L$ e $\alpha$. Destes dois 
parâmetros, projetaremos um P, PI ou PID seguindo as regras da tabela:

\begin{array}{rr} \hline
\text{Controlador} & K &T_i&T_d\\ \hline
\text{P} &1/\alpha&&& \\ \hline
\text{PI} &0.9/\alpha&3L&& \\ \hline
\text{PID} &1.2/\alpha&2L&L/2& \\ \hline
\end{array}

E a seguinte função transferência:
\begin{align}
\frac{\Delta(s)}{E(s)} = K_c \left( 1 + \frac{1}{T_i s} + T_d s \right)
\end{align}

Para encontrar estes parâmetros, revisitaremos a resposta ao degrau 
e escolheremos o ponto de inflexão para traçar a tangente.

In [5]:
time, output = plot_step_response(plant_tf, 
                                  title='Plant step response',
                                  show_points=True)

FigureCanvasNbAgg()

Escolhemos o quinto ponto como ponto de flexão, e vamos tracejar a linha tangente a ele, através da sua derivada (discreta).

In [6]:
def derivate_around(x,y,index):
    return (y[index] - y[index - 1])/(x[index]- x[index - 1])

def tangent_line(x, y, index):
    return y[index]+derivate_around(x, y, index)*(x - x[index])

inflection_index = 5
plt.figure()
plt.title("Inflection point tangent line")
axes = plt.gca()
axes.set_xlim([0,25])
axes.set_ylim([-0.5,2])
plt.grid(True)
plt.plot(time,output,'b',time,tangent_line(time, output, inflection_index),'--r')
plt.show()

FigureCanvasNbAgg()

Da figura acima obtemos $L$ e $\alpha$, da intersecção da reta tangente e o eixos $x$ e $y$, respectivamente:

\begin{align*}
L=1.12\\
\alpha=0.22
\end{align*}

In [40]:
def ziegler_nichols_constants(l,alpha):
    return (1.2/alpha, 2*l, l/2)

l = 1.12
alpha = 0.22
k, ti, td = ziegler_nichols_constants(l, alpha) 

(k, ti, td)

(5.454545454545454, 2.24, 0.56)

Logo, temos:

\begin{align}
K_c &= 5.45 \\
T_i &= 2.24 \\
T_d &= 0.56 \\
\frac{\Delta(s)}{E(s)} &= 5.45 \left( 1 + \frac{1}{2.24 s} + 0.56 s \right) = \frac{6.83 s^2 + 12.2s +5.45}{2.24 s}
\end{align}

In [100]:
controller_tf = TransferFunction([k*ti*td, k*ti, k],[ti, 0])
g = series(controller_tf, plant_tf)
system = feedback(g, 1)
time, output = plot_step_response(system)

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  This is separate from the ipykernel package so we can avoid doing imports until


FigureCanvasNbAgg()

In [72]:
plt.figure()
plt.title('Nyquist plot')
_ = nyquist_plot(g)

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


FigureCanvasNbAgg()

E então, olharemos métricas em frequência:

In [73]:
gm, pm, _, _ = margin(g)
(gm, pm)

(inf, 17.232154029542244)

Como o pacote "control" não implementa métricas no tempo, teremos que implementá-las. 

In [112]:
def settling_time(system, error=0.05):
    time, output = step_response(system)
    settling_time = None
    for t, out in zip(time, output):
        if abs(1-out) < error:
            if settling_time is None:
                settling_time = t
        else:
                settling_time = None
    return settling_time

def rise_time(system, stop=0, start=1):
    time, output = step_response(system)
    interpolated = interp1d(time, output)
    new_time = np.arange(0, 50, 0.01)
    plt.figure()
    plt.plot(new_time,interpolated(new_time))
    return new_time, interpolated(new_time)

def overshoot(system):
        _, output = step_response(system)
        return max(output)
    
sys_settling = settling_time(system)
sys_overshoot = overshoot(system)
rise_time(system)

FigureCanvasNbAgg()

(array([0.000e+00, 1.000e-02, 2.000e-02, ..., 4.997e+01, 4.998e+01,
        4.999e+01]),
 array([0.        , 0.00519446, 0.01038892, ..., 0.99315578, 0.99322815,
        0.99330051]))

### Desempenho ziegler-nichols sem ajustes
\begin{align}
G_m &= \infty \\
P_m &= 17.23^{\circ} \\
M_p & = 1.48 \\
T_s & = 29.9 s
\end{align}